In [3]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
from PIL import Image 
import cv2
import pickle

ModuleNotFoundError: No module named 'cv2'

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread

In [3]:
data_dir_1 = r'OULP-C1V2_Pack/OULP-C1V2_NormalizedSilhouette(224x224x3)'
Seq = os.listdir(data_dir_1)
print(Seq)
sequences = [each for each in Seq if os.path.isdir(data_dir_1 + '/' + each)]
print(sequences)

['.DS_Store', 'Seq00', 'Seq01']
['Seq00', 'Seq01']


# Read Image

In [9]:
from numpy import genfromtxt
IDList_A_55_probe = genfromtxt('OULP-C1V2_Pack/OULP-C1V2_SubjectIDList(FormatVersion1.0)/IDList_OULP-C1V2-A-75_gallery.csv',
           delimiter=',', dtype=np.int32)

# IDList_A_55_probe = genfromtxt('OULP-C1V2_Pack/OULP-C1V2_SubjectIDList(FormatVersion1.0)/IDList_OULP-C1V2-A-75_probe.csv',
#            delimiter=',', dtype=np.int32)

print('length:',len(IDList_A_55_probe))
for row in IDList_A_55_probe:
    print(row)

length: 3759
[ 24   0 102 133]
[ 32   0  79 108]
[ 35   0  97 128]
[36  0 63 95]
[37  0 49 80]
[ 38   0  92 122]
[ 39   0  81 115]
[ 40   0  92 122]
[ 41   0  88 116]
[ 42   0  89 119]
[ 43   0  86 118]
[ 44   0  93 124]
[ 45   0  80 109]
[ 46   0  92 122]
[ 47   0 120 151]
[48  0 67 98]
[ 49   0 101 133]
[ 50   0  99 129]
[ 51   0 103 133]
[ 52   0  92 121]
[ 53   0  90 119]
[ 54   0 118 151]
[ 55   0 131 164]
[ 56   0  71 103]
[ 57   0 114 145]
[ 58   0  87 117]
[ 59   0  98 129]
[ 60   0 114 145]
[ 61   0 106 136]
[ 62   0 106 134]
[ 63   0 133 165]
[ 64   0 111 141]
[ 65   0  89 120]
[ 66   0  91 126]
[ 67   0 124 157]
[ 68   0  99 129]
[ 69   0  97 126]
[ 70   0  92 120]
[ 71   0 109 141]
[ 72   0 107 137]
[ 73   0 119 150]
[ 74   0 121 154]
[ 75   0  88 120]
[1000001       0     149     178]
[1000002       0     156     187]
[1000003       0      97     121]
[1000004       0      69      99]
[1000006       0     108     139]
[1000007       0      91     119]
[1000008       0     

## confirm probe and gallery have same subjects

In [8]:
x = 0 
for i,j in zip(IDList_A_55_gallery, IDList_A_55_probe):
    if i[0] != j[0]:
        x += 1
print(x)

0


In [10]:
def generate_gait_cycle_list(folder_path, csv_file):
    for subject in csv_file:
        images_list = np.ndarray((subject[3] - subject[2] + 1, 224, 224, 3))
    
        subject_dir = str(subject[0])
        if len(subject_dir) != 7:
            subject_dir = '0' * (7 - len(subject_dir)) + subject_dir
                
        for ii, image_name in enumerate(range(subject[2], subject[3]+1)):        
            image_name = str(image_name)
            if len(image_name) == 2:
                image_name = '0' + image_name
            elif len(image_name) == 1:
                image_name = '00' + image_name
            images_list[ii, :, :, :] = imread(os.path.join(folder_path, subject_dir, '00000{}.png'.format(image_name)))
            
            if ii == 0:
                begin_image = image_name
                
        yield (images_list, subject_dir, begin_image, image_name)

In [ ]:
Or_folder_path = 'OULP-C1V2_Pack/OULP-C1V2_NormalizedSilhouette(224x224x3)/Seq00'
# subject_list = os.listdir(Or_folder_path) 
# subject_list = subject_list[1:]
# save_folder = 'OULP-C1V2_Pack/OULP-deference-(128x88)/Seq00'
image_list_tuple = generate_gait_cycle_list(Or_folder_path, IDList_A_55_probe)

total = []
for ii in range(len(IDList_A_55_probe)):
    next_input = next(image_list_tuple)
    total.append(next_input[0])

In [6]:
total = np.array(total)

In [ ]:
with open(r"gait_data/gallery_75degree_1_375", 'wb') as outfile:
    np.savez(outfile, total)

In [ ]:
codes_list = []

codes = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    for each in sequences:
        print("Starting {} sequence".format(each))
        sequence_path = data_dir_1 + '/' + each
        subjects = os.listdir(sequence_path)
        subjects = np.array(subjects)
        subjects = subjects[1:]
        for subject in subjects:
            subject_path = sequence_path + '/' + subject
            if subject == '0000000':
                files = os.listdir(subject_path)
                files.remove('.DS_Store')
#             elif subject == '0000024':
#                 files = os.listdir(subject_path)
#                 files.remove('.DS_Store')
            else:
                files = os.listdir(subject_path)
            
            for ii, file in enumerate(files, 1):
                img = imread(os.path.join(subject_path, file))
                batch.append(img.reshape((1, 224, 224, 3)))
            
                if ii % batch_size == 0 or ii == len(files):
                    images = np.concatenate(batch)
                
                    feed_dict = {input_: images}
                    codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                    if codes is None:
                        codes = codes_batch
                    else:
                        coeds = np.concatenate((codes, codes_batch))
                    
                    # Reset to start building the next batch
                    batch = []
                    print('{} images processed'.format(ii))

/Users/congcong/Desktop/Gait/OULP-Database/tensorflow_vgg/vgg16.npy
npy file loaded
build model started
build model finished: 0s
Starting Seq00 sequence
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
115 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
117 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
119 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
111 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
120 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images processed
120 images processed
132 images processed
20 images processed
40 images processed
60 images processed
80 images processed
100 images pro